In [1]:
from zmax_datasets.datasets.usleep import USleepDataset
from pathlib import Path
import numpy as np

DATASETS_DIR = Path("/project/4180000.46/sleep_datasets/processed")
DATASET_NAME = "mesa"

dataset = USleepDataset(data_dir=DATASETS_DIR / DATASET_NAME)
print(dataset.n_recordings)
print(dataset.recording_ids)

5
['mesa-sleep-0001', 'mesa-sleep-0002', 'mesa-sleep-0006', 'mesa-sleep-0010', 'mesa-sleep-0012']


In [2]:
sample_recording = dataset.get_recording("mesa-sleep-0001")
print(sample_recording.data_types)

{'PPG_filtered': DataType(channel='PPG_filtered', sampling_rate=128.0), 'PPG_ibi': DataType(channel='PPG_ibi', sampling_rate=128.0), 'PPG_peaks': DataType(channel='PPG_peaks', sampling_rate=128.0), 'PPG_quality': DataType(channel='PPG_quality', sampling_rate=128.0), 'PPG_rate': DataType(channel='PPG_rate', sampling_rate=128.0)}


In [3]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from pathlib import Path
import pandas as pd

# Load all PPG signals
ppg_filtered = sample_recording.read_data_type("PPG_filtered")
ppg_peaks = sample_recording.read_data_type("PPG_peaks")
ppg_rate = sample_recording.read_data_type("PPG_rate")
ppg_quality = sample_recording.read_data_type("PPG_quality")
ppg_ibi = sample_recording.read_data_type("PPG_ibi")

# Create time axis in seconds for the signals
duration_seconds = len(ppg_rate.array.squeeze()) / ppg_rate.sample_rate
time = np.linspace(0, duration_seconds, len(ppg_rate.array.squeeze()))
print(f"Signal duration: {duration_seconds:.1f} seconds ({duration_seconds/60:.1f} minutes)")

2025-10-12 22:36:11.390 | INFO     | zmax_datasets.datasets.base:read_data_type:36 - Reading data type: PPG_filtered
2025-10-12 22:36:11.805 | INFO     | zmax_datasets.datasets.base:read_data_type:36 - Reading data type: PPG_peaks
2025-10-12 22:36:11.991 | INFO     | zmax_datasets.datasets.base:read_data_type:36 - Reading data type: PPG_rate
2025-10-12 22:36:12.339 | INFO     | zmax_datasets.datasets.base:read_data_type:36 - Reading data type: PPG_quality
2025-10-12 22:36:12.497 | INFO     | zmax_datasets.datasets.base:read_data_type:36 - Reading data type: PPG_ibi


Signal duration: 43199.0 seconds (720.0 minutes)


In [4]:
import ipywidgets as widgets
from IPython.display import display
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create widgets for controlling the window
start_slider = widgets.FloatSlider(
    value=0,
    min=0,
    max=duration_seconds - 60,  # Leave room for the window
    step=10,
    description='Start Time (s):',
    style={'description_width': 'initial'},
    layout={'width': '500px'}
)

window_size = widgets.Dropdown(
    options=[('10 seconds', 10), ('30 seconds', 30), ('1 minute', 60), ('5 minutes', 300)],
    value=60,
    description='Window Size:',
    style={'description_width': 'initial'}
)

# Create initial figure
fig = go.FigureWidget(make_subplots(
    rows=4, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    row_heights=[0.4, 0.2, 0.2, 0.2],
    subplot_titles=("PPG Signal and Peaks", "Signal Quality", "Heart Rate", "Inter-Beat Intervals")
))

# Initialize with empty traces
fig.add_trace(go.Scatter(name="PPG Filtered", line=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(name="Peaks", mode='markers', marker=dict(color='red', size=8, symbol='circle')), row=1, col=1)
fig.add_trace(go.Scatter(name="Signal Quality", line=dict(color='purple'), fill='tozeroy'), row=2, col=1)
fig.add_trace(go.Scatter(name="Heart Rate", line=dict(color='orange')), row=3, col=1)
fig.add_trace(go.Scatter(name="Mean HR", line=dict(color='red', dash='dash')), row=3, col=1)
# Add IBI trace and threshold lines
fig.add_trace(go.Scatter(name="IBI", line=dict(color='green')), row=4, col=1)
fig.add_trace(go.Scatter(name="Min Threshold", line=dict(color='red', dash='dash')), row=4, col=1)
fig.add_trace(go.Scatter(name="Max Threshold", line=dict(color='red', dash='dash')), row=4, col=1)

# Update layout
fig.update_layout(
    height=900,
    title=f"PPG Signal Analysis - Recording {str(sample_recording)}",
    showlegend=True,
    template="plotly_white"
)

# Update axes labels
fig.update_yaxes(title_text="Amplitude", row=1, col=1)
fig.update_yaxes(title_text="Quality Score", range=[0, 1], row=2, col=1)
fig.update_yaxes(title_text="Heart Rate (BPM)", range=[0, 200], row=3, col=1)
fig.update_yaxes(title_text="IBI (ms)", range=[0, 2500], row=4, col=1)  # Set range slightly above max threshold
fig.update_xaxes(title_text="Time (seconds)", row=4, col=1)

def update_plot(start_time, window_duration):
    # Calculate indices for the window
    start_idx = int(start_time * ppg_rate.sample_rate)
    end_idx = int((start_time + window_duration) * ppg_rate.sample_rate)
    
    # Create time array for the window
    time_window = np.linspace(start_time, start_time + window_duration, end_idx - start_idx)
    
    # Get signal segments
    filtered_signal = ppg_filtered.array.squeeze()[start_idx:end_idx]
    peaks_signal = ppg_peaks.array.squeeze()[start_idx:end_idx]
    quality_signal = ppg_quality.array.squeeze()[start_idx:end_idx]
    rate_signal = ppg_rate.array.squeeze()[start_idx:end_idx]
    ibi_signal = ppg_ibi.array.squeeze()[start_idx:end_idx]
    
    # Update traces with new data
    with fig.batch_update():
        # Update PPG signal
        fig.data[0].x = time_window
        fig.data[0].y = filtered_signal
        
        # Update peaks
        peaks_idx = np.where(peaks_signal == 1)[0]
        if len(peaks_idx) > 0:
            fig.data[1].x = time_window[peaks_idx]
            fig.data[1].y = filtered_signal[peaks_idx]
        else:
            fig.data[1].x = []
            fig.data[1].y = []
        
        # Update template match
        fig.data[2].x = time_window
        fig.data[2].y = quality_signal
        
        # Update heart rate
        fig.data[3].x = time_window
        fig.data[3].y = rate_signal
        
        # Update mean heart rate
        mean_hr = np.nanmean(rate_signal)
        fig.data[4].x = [time_window[0], time_window[-1]]
        fig.data[4].y = [mean_hr, mean_hr]
        fig.data[4].name = f"Mean HR ({mean_hr:.1f} BPM)"
        
        # Update IBI plot
        # Only plot non-zero IBI values
        valid_ibi_mask = ibi_signal > 0
        valid_times = time_window[valid_ibi_mask]
        valid_ibis = ibi_signal[valid_ibi_mask]
        
        fig.data[5].x = valid_times
        fig.data[5].y = valid_ibis
        
        # Update threshold lines
        fig.data[6].x = [time_window[0], time_window[-1]]  # Min threshold
        fig.data[6].y = [300, 300]  # 300ms threshold
        
        fig.data[7].x = [time_window[0], time_window[-1]]  # Max threshold
        fig.data[7].y = [2000, 2000]  # 2000ms threshold

# Create the interactive plot
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        update_plot(start_slider.value, window_size.value)

# Link the widgets to the update function
start_slider.observe(on_change)
window_size.observe(on_change)

# Display widgets and initial plot
display(widgets.HBox([start_slider, window_size]))
display(fig)

# Initialize the plot
update_plot(start_slider.value, window_size.value)


FigureWidget({
    'data': [{'line': {'color': 'blue'},
              'name': 'PPG Filtered',
              'type': 'scatter',
              'uid': 'b6caf1f4-2649-43c2-b258-20a1467d1486',
              'xaxis': 'x',
              'yaxis': 'y'},
             {'marker': {'color': 'red', 'size': 8, 'symbol': 'circle'},
              'mode': 'markers',
              'name': 'Peaks',
              'type': 'scatter',
              'uid': '6564bf6b-545f-4e85-b5c2-e6d82bb5e060',
              'xaxis': 'x',
              'yaxis': 'y'},
             {'fill': 'tozeroy',
              'line': {'color': 'purple'},
              'name': 'Signal Quality',
              'type': 'scatter',
              'uid': 'fe35521f-815d-415c-b289-6a2a903ac717',
              'xaxis': 'x2',
              'yaxis': 'y2'},
             {'line': {'color': 'orange'},
              'name': 'Heart Rate',
              'type': 'scatter',
              'uid': 'aadaf876-759b-4d4c-927c-64c40f169384',
              'xaxis': 'x